In [ ]:
!pip install qci-client==4.5.0

  Using cached qci_client-4.5.0-py3-none-any.whl.metadata (2.3 kB)
Using cached qci_client-4.5.0-py3-none-any.whl (35 kB)


In [8]:
from google.colab import userdata

In [9]:
import numpy as np

In [16]:
F = np.load("/content/taiwanese_dataset_F.npy")
Q = np.load("/content/taiwanese_dataset_Q.npy")

In [14]:
import importlib.metadata
from pprint import pprint

from qci_client import JobStatus, QciClient

# Requires qci-client v4.5.0+.
print(f"qci-client v{importlib.metadata.version('qci-client')}")

url = "https://api.qci-prod.com"
api_token = userdata.get('API_KEY')
client = QciClient(url=url, api_token=api_token)

print(client.get_allocations()["allocations"]["dirac"])
# This should output something similar to:
# qci-client v4.5.0
# {'metered': True, 'seconds': 600}

qci-client v4.5.0
{'metered': True, 'seconds': 598, 'paid': False}


In [17]:
# Lists to store coefficients and indices
poly_coef = []
poly_indices = []

# Iterate over the lower triangular part (including the diagonal)
n = Q.shape[0]
for i in range(n):
    for j in range(i + 1):  # Only iterate over lower triangular part
        poly_coef.append(float(Q[i, j]))  # Add value to poly_coef
        poly_indices.append([j + 1, i + 1])  # Adjust indices by adding 1

# Output results
print("poly_coef:", poly_coef)
print("poly_indices:", poly_indices)


poly_coef: [6.353032581185552, 1.3075637603359445, 6.334760416110274, 2.1662081427089914, 1.497990932622855, 6.339489485810658, 0.20783489775905029, 0.20867611826065602, 0.20698217726786883, 6.3868310815674745, 0.20049386999308005, 0.20456418413254962, 0.20369767635339286, 5.257662300785251, 6.385428744353433, 0.49890346895881077, 0.4978356713302219, 0.5020841868725467, 0.5103515229779472, 0.5079736022100532, 6.345825947574305, 0.6617671277243704, 0.8947581977945775, 0.6592976528941112, 0.31917188400094876, 0.3181961035953753, 0.970209311811546, 6.372235594667311, 0.6037684571042994, 0.8827043170012567, 0.6495746879248561, 0.3121162394223522, 0.31025546632597445, 0.8818910984003558, 2.4559625309427737, 6.362725984749762, 0.14705345486475618, 0.20234830152748406, 0.14056526835836447, 0.031254915625292146, 0.03153771953870521, 0.0849218221023964, 0.39059512385947937, 0.38375552712430494, 6.241639038824792, 0.7001221697689881, 0.8249672744404597, 0.7601340417875386, 0.33576137289725017, 0

In [19]:
# Let's consider a simple Hamiltonian problem
poly_indices_cts_problem = poly_indices
poly_coefs_cts_problem = poly_coef
data_cts_problem = [{"idx": idx, "val": val} for idx, val in zip(poly_indices_cts_problem, poly_coefs_cts_problem)]
file_cts_problem = {
    "file_name": "dirac_3_continuous_variable_example",
    "file_config": {
        "polynomial": {
            "num_variables": 95,
            "min_degree": 1,
            "max_degree": 2,
            "data": data_cts_problem,
        }
    }
}

file_response_cts_problem = client.upload_file(file=file_cts_problem)
file_response_cts_problem
# This should output something similar to:
# {'file_id': '670ff26b5e0855263226e27f'}

{'file_id': '67f36b2a5e0855263227e82c'}

In [20]:
# Build the job body to be submitted to the QCi API.
# Customize the job body for the Parkinson dataset and QPFS method.

job_body_cts_problem = client.build_job_body(
    job_type='sample-hamiltonian',
    job_name='QPFS_taiwanese_Hamiltonian_Job',  # Custom name reflecting Parkinson dataset and feature selection task
    job_tags=['QPFS', 'Taiwanese_banckruptsy', 'Hamiltonian'],  # Tags relevant to the project
    job_params={
        'device_type': 'dirac-3',
        'relaxation_schedule': 1,  # Adjust if necessary based on the problem
        'sum_constraint': 1,  # Adjust if needed for specific constraints
        # Additional parameters specific to your QPFS method can be added here
    },
    polynomial_file_id=file_response_cts_problem['file_id'],  # Use the file ID from previous step
)

# Print the job body for confirmation
pprint(job_body_cts_problem)


{'job_submission': {'device_config': {'dirac-3_normalized_qudit': {'relaxation_schedule': 1,
                                                                   'sum_constraint': 1}},
                    'job_name': 'QPFS_taiwanese_Hamiltonian_Job',
                    'job_tags': ['QPFS',
                                 'Taiwanese_banckruptsy',
                                 'Hamiltonian'],
                    'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '67f36b2a5e0855263227e82c'}}}}


In [21]:
# Submit the job and await the result.
job_response_cts_problem = client.process_job(job_body=job_body_cts_problem)

# Check if the job completed successfully.
if job_response_cts_problem["status"] != JobStatus.COMPLETED.value:
    raise Exception(f"Job failed with status: {job_response_cts_problem['status']}")

# Retrieve all solutions and energies
solutions = job_response_cts_problem['results']['solutions']
energies = job_response_cts_problem['results']['energies']

# Iterate through solutions and print the corresponding energy
for sol, energy in zip(solutions, energies):
    print(f"Solution: {sol} with energy: {energy}")

# This should output something similar to:
# 2024-05-15 10:59:49 - Dirac allocation balance = 600 s
# 2024-05-15 10:59:49 - Job submitted: job_id='6644ea05d448b017e54f9663'
# 2024-05-15 10:59:49 - QUEUED
# 2024-05-15 10:59:52 - RUNNING
# 2024-05-15 11:00:46 - COMPLETED
# 2024-05-15 11:00:48 - Dirac allocation balance = 599 s
# solution: 0.4174435, 0.5825538, 0, 0] with energy: 0.8749975

2025-04-07 06:06:49 - Dirac allocation balance = 598 s
2025-04-07 06:06:49 - Job submitted: job_id='67f36b796f2d98bd02a34036'
2025-04-07 06:06:49 - QUEUED
2025-04-07 06:06:52 - RUNNING
2025-04-07 06:06:57 - COMPLETED
2025-04-07 06:06:59 - Dirac allocation balance = 597 s
Solution: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0001204, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.000534, 7e-07, 0, 0, 0, 5.73e-05, 0, 0.0002169, 0, 0, 0, 0, 3.85e-05, 0.0001637, 0, 0, 0, 0.0009036, 0, 0, 0, 1.9e-06, 0, 0, 0, 1e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5864629, 0, 0.0001297, 0, 0, 0, 0, 0, 0, 0.4113702, 0] with energy: 1.4e-06


In [22]:
print(job_response_cts_problem)

# This should output something similar to:
# {'job_info': {'job_id': '6750b4b4324e163bf2d89395',
#               'job_result': {'device_usage_s': 1,
#                              'file_id': '6750b4b65e08552632272819'},
#               'job_status': {'completed_at_rfc3339nano': '2024-12-04T19:59:50.632Z',
#                              'queued_at_rfc3339nano': '2024-12-04T19:59:48.302Z',
#                              'running_at_rfc3339nano': '2024-12-04T19:59:48.679Z',
#                              'submitted_at_rfc3339nano': '2024-12-04T19:59:48.302Z'},
#               'job_submission': {'device_config': {'dirac-3_normalized_qudit': {'num_samples': 1,
#                                                                                 'relaxation_schedule': 1,
#                                                                                 'sum_constraint': 1}},
#                                  'job_name': 'test_continuous_variable_hamiltonian_job',
#                                  'job_tags': ['tag1', 'tag2'],
#                                  'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '6750b4ab5e08552632272817'}}}},
#  'results': {'counts': [1],
#              'energies': [0.8749975],
#              'solutions': [[0.4174435, 0.5825538, 0, 0]]},
#  'status': 'COMPLETED'

{'job_info': {'job_id': '67f36b796f2d98bd02a34036', 'job_submission': {'job_name': 'QPFS_taiwanese_Hamiltonian_Job', 'job_tags': ['QPFS', 'Taiwanese_banckruptsy', 'Hamiltonian'], 'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '67f36b2a5e0855263227e82c'}}, 'device_config': {'dirac-3_normalized_qudit': {'num_samples': 1, 'relaxation_schedule': 1, 'sum_constraint': 1}}}, 'job_status': {'submitted_at_rfc3339nano': '2025-04-07T06:06:49.328Z', 'queued_at_rfc3339nano': '2025-04-07T06:06:49.328Z', 'running_at_rfc3339nano': '2025-04-07T06:06:49.628Z', 'completed_at_rfc3339nano': '2025-04-07T06:06:56.097Z'}, 'job_result': {'file_id': '67f36b805e0855263227e82e', 'device_usage_s': 1}}, 'status': 'COMPLETED', 'results': {'counts': [1], 'energies': [1.4e-06], 'solutions': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1e-07, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0001204, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.000534, 7e-07, 0, 0, 0, 5.73e-05, 0, 0.0

In [23]:
solutions

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1e-07,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.0001204,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.000534,
  7e-07,
  0,
  0,
  0,
  5.73e-05,
  0,
  0.0002169,
  0,
  0,
  0,
  0,
  3.85e-05,
  0.0001637,
  0,
  0,
  0,
  0.0009036,
  0,
  0,
  0,
  1.9e-06,
  0,
  0,
  0,
  1e-07,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.5864629,
  0,
  0.0001297,
  0,
  0,
  0,
  0,
  0,
  0,
  0.4113702,
  0]]

In [24]:
cont = np.save("solution.npy", solutions)

In [26]:
np.sum(solutions)

1.0